In [55]:
project_name = ''

StatementMeta(new, 97, 1, Finished, Available)

In [56]:
# access to the data

ADLS_PATH = "abfss://container@storage_account.dfs.core.windows.net/fb-scraper/" + project_name # TODO put your proper path
posts_file = spark.read.option('header','true').option('delimiter', ',').option("multiline", 'true').option("quote", '"').option("escape", '"').csv(ADLS_PATH + "/bronze/" + project_name + '_posts.csv')
comments_file = spark.read.option('header','true').option('delimiter', ',').option("multiline", 'true').option("quote", '"').option("escape", '"').csv(ADLS_PATH + "/bronze/" + project_name + '_comments.csv')


StatementMeta(new, 97, 2, Finished, Available)

In [62]:
# select column for the translation

comments = comments_file.select("comment_text", "comment_id", "post_id").collect()
posts = posts_file.select("text", "post_id").collect()

StatementMeta(new, 97, 8, Finished, Available)

In [63]:
import requests, uuid, json

def translate_to_en(text: str):
    subscription_key = ""
    endpoint = "https://api.cognitive.microsofttranslator.com"
    location = "northeurope"
    path = '/translate'
    constructed_url = endpoint + path

    constructed_url = endpoint + path

    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key,
        'Ocp-Apim-Subscription-Region': location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }

    body = [{
        'text': text
    }]
    params = {
        'api-version': '3.0',
        # 'from': 'pl',
        'to': 'en'
    }
    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()
    r = json.dumps(response, sort_keys=True, ensure_ascii=False, indent=4, separators=(',', ': '))
    return r

StatementMeta(new, 97, 9, Finished, Available)

In [67]:
# translate comments

original_comments = []
languages_detected = []
languages_detected_score = []
translations = []
c_ids = []
p_ids = []

for comment in comments:
    c = comment.asDict()["comment_text"].replace("\n", "")
    if c:
        c_id_ = comment.asDict()["comment_id"]
        p_id_ = comment.asDict()["post_id"]
        c_ids.append(c_id_)
        p_ids.append(p_id_)
        original_comments.append(c)
        res = json.loads(translate_to_en(c))[0]
        languages_detected.append(res["detectedLanguage"]["language"])
        languages_detected_score.append(res["detectedLanguage"]["score"])
        translations.append(res["translations"][0]["text"])

StatementMeta(new, 97, 13, Finished, Available)

In [22]:
final_results = sqlContext.createDataFrame(zip(original_comments, languages_detected, languages_detected_score, translations, c_ids, p_ids), schema=['Original', 'Detected language', 'Detected language score', 'Translation', 'CommentID', 'PostID'])
final_results.write.format("csv").option("header","true").option("sep",",").mode("overwrite").save(ADLS_PATH + "/silver/" + 'comments_en.csv')
# display(final_results)

StatementMeta(new, 84, 22, Finished, Available)

In [24]:
# translate posts

original_posts = []
languages_detected = []
languages_detected_score = []
translations = []
p_ids = []


for post in posts:
    c = post.asDict()["text"]
    if c:
        original_posts.append(c)
        p_id_ = post.asDict()["post_id"]
        p_ids.append(p_id_)
        res = json.loads(translate_to_en(c))[0]
        languages_detected.append(res["detectedLanguage"]["language"])
        languages_detected_score.append(res["detectedLanguage"]["score"])
        translations.append(res["translations"][0]["text"])

final_results = sqlContext.createDataFrame(zip(original_posts, languages_detected, languages_detected_score, translations, p_ids), schema=['Original', 'Detected language', 'Detected language score', 'Translation', 'PostID'])

final_results.write.format("csv").option("header","true").option("sep",",").mode("overwrite").save(ADLS_PATH + '/silver/' + 'posts_en.csv')

# display(final_results)

StatementMeta(new, 84, 24, Finished, Cancelled)